In [67]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from statsmodels import api
from scipy import stats
from scipy.optimize import minimize 
import json


In [45]:
def cleanData(df):
    # Remove unnecessary columns
    df = df.drop('Link', axis=1)
    df['c_0'] = 0


    # Calc c_0      
    c_0 = {
        2010: df.loc[df['Year'] == 2010, 'Cited by'].mean(),
        2011: df.loc[df['Year'] == 2011, 'Cited by'].mean(),
        2012: df.loc[df['Year'] == 2012, 'Cited by'].mean(),
        2013: df.loc[df['Year'] == 2013, 'Cited by'].mean(),
        2014: df.loc[df['Year'] == 2014, 'Cited by'].mean(),
        2015: df.loc[df['Year'] == 2015, 'Cited by'].mean()
    }
    for index, row in df.iterrows():
        df['c_0'][index] = c_0[row['Year']]
  
    # Calc c_f
    df['c_f'] = df['Cited by'] / df['c_0']

    return df




In [16]:
def log_normal_func(mu, sigma):
    f = 1 / (sigma*mu*np.sqrt(2*np.pi)) * np.exp(-np.power((np.log(mu)-mu),2) / (2*np.power(sigma,2)))
    return f

In [17]:
def get_opti_fun(df):
   # MLE function
   def MLE_Norm(parameters):
      # extract parameters
      mu, sigma = parameters
      # Calculate the log-likelihood for normal distribution
      LL = np.sum(stats.norm.logpdf(df['c_f'], loc=mu, scale=sigma))
      # Calculate the negative log-likelihood
      neg_LL = -1*LL
      return neg_LL
   return MLE_Norm

In [28]:
ai_ml = pd.read_csv('AI_ML.csv')
com_net = pd.read_csv('Communications_Networking.csv')
con_alg = pd.read_csv('ControlSystems_Algorithms.csv')
edge_com = pd.read_csv('EdgeComputing.csv')
loc_map = pd.read_csv('Localisation_Mapping.csv')
rob = pd.read_csv('Robotics.csv')
sen_fus = pd.read_csv('sensor_fusion.csv')
sen_tec = pd.read_csv('sensor_technology.csv')
sim_mod = pd.read_csv('Simulation_Modeling.csv')
swarm_mult = pd.read_csv('Swarming_MultiAgent.csv')

dataSets = [ai_ml, com_net, con_alg, edge_com, loc_map, rob, sen_fus, sen_tec, sim_mod, swarm_mult]

In [64]:
def MLE_estimation(dataSets):

    parameterDict = {}
    key_names = ["ai_ml", "com_net", "con_alg", "edge_com", "loc_map", "rob", "sen_fus", "sen_tec", "sim_mod", "swarm_mult"]
    initParams = [1,1]
    
    for index, df in enumerate(dataSets):
        df = cleanData(df)

        mle = minimize(get_opti_fun(df), initParams, method='Nelder-Mead')
        mu = mle.x[0]
        sigma = mle.x[1]
        parameterDict.update({key_names[index]: {'mu': mu}})
        parameterDict[key_names[index]].update({'sigma': sigma})

    return parameterDict

        

In [65]:
parameters = MLE_estimation(dataSets)

/var/folders/9r/w8p_5y0s0913xqcffg4y86sr0000gn/T/ipykernel_7681/984332791.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['c_0'][index] = c_0[row['Year']]
/var/folders/9r/w8p_5y0s0913xqcffg4y86sr0000gn/T/ipykernel_7681/984332791.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['c_0'][index] = c_0[row['Year']]
/var/folders/9r/w8p_5y0s0913xqcffg4y86sr0000gn/T/ipykernel_7681/984332791.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

In [72]:
with open("distribution_params.json", "w") as write_file:
    json.dump(parameters, write_file, indent=4)